# Preprocess Raw Data

In [1]:
library(reshape)
library(plyr)


Attaching package: ‘plyr’


The following objects are masked from ‘package:reshape’:

    rename, round_any




In [2]:
horizontaldf2verticaldf <- function(horizontal.df,value_column_name){
    colnames(horizontal.df) <- c(c("Province.State","Country.Region","Lat","Long"),seq(as.Date("2020-01-22"), length = ncol(global_recovered.df)-4, by = "days"))
    vertical.df <- melt(horizontal.df,
             id=c("Province.State","Country.Region","Lat","Long"))
    colnames(vertical.df) <- c(c("Province.State","Country.Region","Lat","Long"),"Date",value_column_name)
    vertical.df$Date <- as.Date(as.integer(vertical.df$Date),origin="21/01/2020", format="%d/%m/%Y")
    return(vertical.df)
}

In [3]:
# Given a dataframe df of data and a list of country / province pairs,
# this functions adds a row to df for each element of pairs not present in df.
# This is used because we have regional data for deaths and confirmed cases,
# but not for recovery in some countries, and thus the merge discard these countries
add_missing_rows <- function(df, pairs, value_column_name) {
    for (i in nrow(pairs)) {
        confirmed_row <- df[(df$Country.Region == pairs[i, "Country.Region"]) &
                            (df$Province.State == pairs[i, "Province.State"]),]
        if (nrow(confirmed_row) == 0) {
            new_row <- df[df$Country.Region == pairs[i, "Country.Region"],][1,]
            new_row[,value_column_name] <- NA
            new_row[,"Province.State"] <- pairs[i, "Province.State"]
            df <- rbind(df, new_row)
        }
    }
    return(df)
}

## Read data

In [36]:
italy.df <- read.csv("data/Italy/dpc-covid19-ita-regioni.csv")

In [37]:
global_confirmed.df <- read.csv("data/Global_JohnsHopkins/time_series_covid19_confirmed_global.csv")
global_deaths.df <- read.csv("data/Global_JohnsHopkins/time_series_covid19_deaths_global.csv")
global_recovered.df <- read.csv("data/Global_JohnsHopkins/time_series_covid19_recovered_global.csv")

## Check data structure

In [38]:
head(italy.df)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_attualmente_positivi,nuovi_attualmente_positivi,dimessi_guariti,deceduti,totale_casi,tamponi,note_it,note_en
,<fct>,<fct>,<int>,<fct>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<fct>,<fct>
1,2020-02-24T18:00:00,ITA,13,Abruzzo,42.35122,13.39844,0,0,0,0,0,0,0,0,0,5,,
2,2020-02-24T18:00:00,ITA,17,Basilicata,40.63947,15.80515,0,0,0,0,0,0,0,0,0,0,,
3,2020-02-24T18:00:00,ITA,4,P.A. Bolzano,46.49933,11.35662,0,0,0,0,0,0,0,0,0,1,,
4,2020-02-24T18:00:00,ITA,18,Calabria,38.90598,16.59440,0,0,0,0,0,0,0,0,0,1,,
5,2020-02-24T18:00:00,ITA,15,Campania,40.83957,14.25085,0,0,0,0,0,0,0,0,0,10,,
6,2020-02-24T18:00:00,ITA,8,Emilia Romagna,44.49437,11.34172,10,2,12,6,18,18,0,0,18,148,,


In [39]:
head(global_confirmed.df)

,Province.State,Country.Region,Lat,Long,X1.22.20,X1.23.20,X1.24.20,X1.25.20,X1.26.20,X1.27.20,...,X3.18.20,X3.19.20,X3.20.20,X3.21.20,X3.22.20,X3.23.20,X3.24.20,X3.25.20,X3.26.20,X3.27.20
,<fct>,<fct>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,22,22,24,24,40,40,74,84,94,110
2,,Albania,41.1533,20.1683,0,0,0,0,0,0,...,59,64,70,76,89,104,123,146,174,186
3,,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,74,87,90,139,201,230,264,302,367,409
4,,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,39,53,75,88,113,133,164,188,224,267
5,,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,0,0,1,2,2,3,3,3,4,4
6,,Antigua and Barbuda,17.0608,-61.7964,0,0,0,0,0,0,...,1,1,1,1,1,3,3,3,7,7


In [40]:
head(global_deaths.df)

,Province.State,Country.Region,Lat,Long,X1.22.20,X1.23.20,X1.24.20,X1.25.20,X1.26.20,X1.27.20,...,X3.18.20,X3.19.20,X3.20.20,X3.21.20,X3.22.20,X3.23.20,X3.24.20,X3.25.20,X3.26.20,X3.27.20
,<fct>,<fct>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,0,0,0,0,1,1,1,2,4,4
2,,Albania,41.1533,20.1683,0,0,0,0,0,0,...,2,2,2,2,2,4,5,5,6,8
3,,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,7,9,11,15,17,17,19,21,25,26
4,,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,0,0,0,0,1,1,1,1,3,3
5,,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,,Antigua and Barbuda,17.0608,-61.7964,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
head(global_recovered.df)

,Province.State,Country.Region,Lat,Long,X1.22.20,X1.23.20,X1.24.20,X1.25.20,X1.26.20,X1.27.20,...,X3.18.20,X3.19.20,X3.20.20,X3.21.20,X3.22.20,X3.23.20,X3.24.20,X3.25.20,X3.26.20,X3.27.20
,<fct>,<fct>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,1,1,1,1,1,1,1,2,2,2
2,,Albania,41.1533,20.1683,0,0,0,0,0,0,...,0,0,0,2,2,2,10,17,17,31
3,,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,12,32,32,32,65,65,24,65,29,29
4,,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
5,,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,,Antigua and Barbuda,17.0608,-61.7964,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Global
### Transform horizontal to vertical

In [42]:
global_confirmed_vertical.df <- horizontaldf2verticaldf(global_confirmed.df,"Confirmed")
global_deaths_vertical.df <- horizontaldf2verticaldf(global_deaths.df,"Deaths")
global_recovered_vertical.df <- horizontaldf2verticaldf(global_recovered.df,"Recovered")

In [43]:
head(global_confirmed_vertical.df)

,Province.State,Country.Region,Lat,Long,Date,Confirmed
,<fct>,<fct>,<dbl>,<dbl>,<date>,<int>
1,,Afghanistan,33.0000,65.0000,2020-01-22,0
2,,Albania,41.1533,20.1683,2020-01-22,0
3,,Algeria,28.0339,1.6596,2020-01-22,0
4,,Andorra,42.5063,1.5218,2020-01-22,0
5,,Angola,-11.2027,17.8739,2020-01-22,0
6,,Antigua and Barbuda,17.0608,-61.7964,2020-01-22,0


In [44]:
head(global_recovered_vertical.df)

,Province.State,Country.Region,Lat,Long,Date,Recovered
,<fct>,<fct>,<dbl>,<dbl>,<date>,<int>
1,,Afghanistan,33.0000,65.0000,2020-01-22,0
2,,Albania,41.1533,20.1683,2020-01-22,0
3,,Algeria,28.0339,1.6596,2020-01-22,0
4,,Andorra,42.5063,1.5218,2020-01-22,0
5,,Angola,-11.2027,17.8739,2020-01-22,0
6,,Antigua and Barbuda,17.0608,-61.7964,2020-01-22,0


In [45]:
head(global_deaths_vertical.df)

,Province.State,Country.Region,Lat,Long,Date,Deaths
,<fct>,<fct>,<dbl>,<dbl>,<date>,<int>
1,,Afghanistan,33.0000,65.0000,2020-01-22,0
2,,Albania,41.1533,20.1683,2020-01-22,0
3,,Algeria,28.0339,1.6596,2020-01-22,0
4,,Andorra,42.5063,1.5218,2020-01-22,0
5,,Angola,-11.2027,17.8739,2020-01-22,0
6,,Antigua and Barbuda,17.0608,-61.7964,2020-01-22,0


### Merge data

In [46]:
global_confirmed_vertical.df[,"Country.Region"] <- as.character(global_confirmed_vertical.df[,"Country.Region"])
global_confirmed_vertical.df[,"Province.State"] <- as.character(global_confirmed_vertical.df[,"Province.State"])
global_deaths_vertical.df[,"Country.Region"] <- as.character(global_deaths_vertical.df[,"Country.Region"])
global_deaths_vertical.df[,"Province.State"] <- as.character(global_deaths_vertical.df[,"Province.State"])
global_recovered_vertical.df[,"Country.Region"] <- as.character(global_recovered_vertical.df[,"Country.Region"])
global_recovered_vertical.df[,"Province.State"] <- as.character(global_recovered_vertical.df[,"Province.State"])

In [47]:
# Get the list of all country / province pairs in all global dataframes (not necessarily present in all of them)
pairs <- unique(rbind(
        global_confirmed_vertical.df[,c("Country.Region", "Province.State")],
        global_deaths_vertical.df[,c("Country.Region", "Province.State")],
        global_recovered_vertical.df[,c("Country.Region", "Province.State")]))

global_confirmed_vertical.df <- add_missing_rows(global_confirmed_vertical.df, pairs, "Confirmed")
global_deaths_vertical.df <- add_missing_rows(global_deaths_vertical.df, pairs, "Deaths")
global_recovered_vertical.df <- add_missing_rows(global_recovered_vertical.df, pairs, "Recovered")

In [48]:
global_merged.df <- merge(global_confirmed_vertical.df,
      merge(global_recovered_vertical.df,global_deaths_vertical.df,
      by=c("Province.State","Country.Region","Date")),
      by=c("Province.State","Country.Region","Date"))

In [49]:
"Canada" %in% global_merged.df$Country.Region

[1] TRUE

## Italy

### Translate columns

In [50]:
colnames(italy.df) <- c("Date","Country.Region","RegionCode","Province.State","Lat","Long","HospitalizedWSymptoms","ICU","TotalHospitalized","HomeIsolation","Confirmed","DailyConfirmed","Recovered","Deaths","Total","Tests")
italy.df$Country.Region <- "Italy"
italy.df$Date <- as.Date(italy.df$Date)
head(italy.df)

,Date,Country.Region,RegionCode,Province.State,Lat,Long,HospitalizedWSymptoms,ICU,TotalHospitalized,HomeIsolation,Confirmed,DailyConfirmed,Recovered,Deaths,Total,Tests,NA,NA
,<date>,<chr>,<int>,<fct>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<fct>,<fct>
1,2020-02-24,Italy,13,Abruzzo,42.35122,13.39844,0,0,0,0,0,0,0,0,0,5,,
2,2020-02-24,Italy,17,Basilicata,40.63947,15.80515,0,0,0,0,0,0,0,0,0,0,,
3,2020-02-24,Italy,4,P.A. Bolzano,46.49933,11.35662,0,0,0,0,0,0,0,0,0,1,,
4,2020-02-24,Italy,18,Calabria,38.90598,16.59440,0,0,0,0,0,0,0,0,0,1,,
5,2020-02-24,Italy,15,Campania,40.83957,14.25085,0,0,0,0,0,0,0,0,0,10,,
6,2020-02-24,Italy,8,Emilia Romagna,44.49437,11.34172,10,2,12,6,18,18,0,0,18,148,,


### Rearrange columns and drop useless columns

In [51]:
italy.df <- italy.df[,c(4,2,5,6,1,11,13,14,7,8,9,10,12,15,16)]

### Merge with global dataset and fill with NA

In [52]:
output.df <- rbind.fill(italy.df,global_merged.df)

### Sort data by date and then by country in alphabetic order

In [53]:
output.df <- output.df[order(output.df$Country.Region, output.df$Date),]
head(output.df)

,Province.State,Country.Region,Lat,Long,Date,Confirmed,Recovered,Deaths,HospitalizedWSymptoms,ICU,TotalHospitalized,HomeIsolation,DailyConfirmed,Total,Tests,Lat.x,Long.x,Lat.y,Long.y
,<chr>,<chr>,<dbl>,<dbl>,<date>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
715,,Afghanistan,33,65,2020-01-22,0,0,0,NA,NA,NA,NA,NA,NA,NA,33,65,33,65
716,,Afghanistan,33,65,2020-01-23,0,0,0,NA,NA,NA,NA,NA,NA,NA,33,65,33,65
717,,Afghanistan,33,65,2020-01-24,0,0,0,NA,NA,NA,NA,NA,NA,NA,33,65,33,65
718,,Afghanistan,33,65,2020-01-25,0,0,0,NA,NA,NA,NA,NA,NA,NA,33,65,33,65
719,,Afghanistan,33,65,2020-01-26,0,0,0,NA,NA,NA,NA,NA,NA,NA,33,65,33,65
720,,Afghanistan,33,65,2020-01-27,0,0,0,NA,NA,NA,NA,NA,NA,NA,33,65,33,65


In [54]:
saveRDS(output.df, "data/COVID19_Global_Italy.Rdata", version = 2)

## Preprocess extra country data

In [2]:
restrictions.df <- read.csv("data/Kaggle_CountryInfo/restrictions_columnwise_updated.csv")
countryinfo.df <- read.csv("data/Kaggle_CountryInfo/covid19countryinfo.csv")

In [3]:
head(restrictions.df)

,Country.Continent,Country.Region,Province.State,Date.Schools,Date.Public.Places,Date.Gatherings,Date.Stay.at.Home,Date.Lockdown,Date.Non.essential,Gatherings.limit,mandatory,notes
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>
1,,Wyoming,Wyoming,2020-03-15T00:00:00.000000000,2020-03-19T00:00:00.000000000,,,,,0,"Yes,Yes",","
2,,Wisconsin,Wisconsin,2020-03-18T00:00:00.000000000,2020-03-17T00:00:00.000000000,2020-03-20T00:00:00.000000000,,,2020-03-24T00:00:00.000000000,10,"Yes,Yes,Yes,Yes","Restriction strengthened to order closure of all non-essential business on 3/24,,,"
3,,West Virginia,West Virginia,2020-03-17T00:00:00.000000000,2020-03-17T00:00:00.000000000,,2020-03-24T00:00:00.000000000,,,0,"Yes,Yes,Yes",",,"
4,,Washington DC,Washington DC,2020-03-16T00:00:00.000000000,2020-03-15T00:00:00.000000000,,,,,0,"Yes,Yes",","
5,,Washington,Washington,2020-03-15T00:00:00.000000000,2020-03-15T00:00:00.000000000,,2020-03-23T00:00:00.000000000,,,0,"Yes,Yes,Yes",",,"
6,,Virginia,Virginia,2020-03-13T00:00:00.000000000,2020-03-17T00:00:00.000000000,2020-03-24T00:00:00.000000000,,,2020-03-24T00:00:00.000000000,10,"Yes,Yes,Yes,Yes",",Restriction on bars and restaurants not toal, limits to 10 patrons,,"


In [4]:
head(countryinfo.df)

,country,pop,tests,testpop,density,medianage,urbanpop,quarantine,schools,restrictions,⋯,sex0,sex14,sex25,sex54,sex64,sex65plus,sexratio,lung,femalelung,malelung
,<fct>,<fct>,<int>,<dbl>,<int>,<int>,<int>,<fct>,<fct>,<fct>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Afghanistan,"38,928,346",NA,NA,60,18,25,,,,⋯,1.05,1.03,1.03,1.03,0.97,0.85,1.03,37.62,36.31,39.33
2,Albania,"2,877,797",NA,NA,105,36,63,,,,⋯,1.08,1.11,1.09,0.93,0.95,0.87,0.98,11.67,7.02,17.04
3,Algeria,"43,851,044",NA,NA,18,29,73,,,,⋯,1.05,1.05,1.05,1.03,1.01,0.89,1.03,8.77,5.03,12.81
4,Andorra,"77,265",NA,NA,164,45,88,,,,⋯,1.07,1.05,1.08,1.05,1.15,1.02,1.06,NA,NA,NA
5,Antigua and Barbuda,"97,929",NA,NA,223,34,26,,,,⋯,1.05,1.03,0.99,0.84,0.82,0.76,0.90,11.76,7.67,18.78
6,Argentina,"45,195,774",NA,NA,17,32,93,3/20/2020,,,⋯,1.05,1.06,1.05,1.00,0.94,0.71,0.98,29.27,20.16,42.59


### Rename columns for consistency

In [5]:
names(restrictions.df)[names(restrictions.df) == "Date.Public.Places"] <- "Date.Public Places"
names(restrictions.df)[names(restrictions.df) == "Date.Stay.at.Home"] <- "Date.Stay at Home"
names(restrictions.df)[names(restrictions.df) == "Date.Non.essential"] <- "Date.Non-essential"
names(restrictions.df)[names(restrictions.df) == "notes"] <- "Notes"
names(countryinfo.df)[names(countryinfo.df) == "country"] <- "Country.Region"

### Remove unused columns
We will have this data after we merge it with restrictions.df

In [6]:
countryinfo.df[c("quarantine", "schools", "restrictions")] <- NULL
restrictions.df[c("Notes", "Gatherings.limit")] <- NULL

### Convert all date column to proper R date type

In [7]:
for (colname in c("Date.Schools", "Date.Public Places", "Date.Gatherings", "Date.Stay at Home", "Date.Lockdown", "Date.Non-essential")) {
    dates <- as.character(restrictions.df[,colname])
    dates[dates == ""] <- NA
    restrictions.df[,colname] <- as.Date(dates)
}

### Merge country extra data and lockdown dates

We have two sources of data for stay at home / lockdown dates, we will keep in priority the ones from the online dataset.

In [8]:
restrictions.df[is.na(restrictions.df[,"Date.Stay at Home"]), "Date.Stay at Home"] <- restrictions.df[is.na(restrictions.df[,"Date.Stay at Home"]), "Date.Lockdown"]
restrictions.df[,"Date.Lockdown"] <- NULL

In [9]:
country_info_output.df <- merge(restrictions.df, countryinfo.df, by=c("Country.Region"))

In [10]:
restrictions.df

Country.Continent,Country.Region,Province.State,Date.Schools,Date.Public Places,Date.Gatherings,Date.Stay at Home,Date.Non-essential,mandatory
<fct>,<fct>,<fct>,<date>,<date>,<date>,<date>,<date>,<fct>
,Wyoming,Wyoming,2020-03-15,2020-03-19,NA,NA,NA,"Yes,Yes"
,Wisconsin,Wisconsin,2020-03-18,2020-03-17,2020-03-20,NA,2020-03-24,"Yes,Yes,Yes,Yes"
,West Virginia,West Virginia,2020-03-17,2020-03-17,NA,2020-03-24,NA,"Yes,Yes,Yes"
,Washington DC,Washington DC,2020-03-16,2020-03-15,NA,NA,NA,"Yes,Yes"
,Washington,Washington,2020-03-15,2020-03-15,NA,2020-03-23,NA,"Yes,Yes,Yes"
,Virginia,Virginia,2020-03-13,2020-03-17,2020-03-24,NA,2020-03-24,"Yes,Yes,Yes,Yes"
,Vietnam,Vietnam,2020-02-12,NA,NA,2020-02-13,NA,"Yes,Yes"
,Vermont,Vermont,2020-03-18,2020-03-16,NA,2020-03-24,NA,"Yes,Yes,Yes"
South America,Venezuela,Venezuela,NA,NA,NA,2020-03-17,NA,Yes


In [11]:
saveRDS(country_info_output.df, "data/COVID19_Country_Info.Rdata", version = 2)